In [158]:
import pandas as pd
import plotly.express as px
import re

df=pd.read_csv('./nasa.csv')
df.drop(df.iloc[:,[0,1,3,5,6,7,8,9,10,13,15,16,17,19,20,21,38]], axis=1, inplace=True)

In [159]:
years=[]
for i in df['Close Approach Date'].values:
    years.extend(re.findall('\d+', i))

data=[]
for i in range(14061):
    if len(years[i])>3:
        data.append(years[i]) 

df['Close Approach Date']=data

In [160]:
years2=[]
for i in df['Orbit Determination Date'].values:
    years2.extend(re.findall('\d+', i))

data2=[]
for i in range(28121):
    if len(years2[i])==4:
        data2.append(years2[i])

df['Orbit Determination Date']=data2

In [161]:
df['Close Approach Date']=df['Close Approach Date'].astype(int)
df['Orbit Determination Date']=df['Orbit Determination Date'].astype(int)

In [162]:
encoded1=pd.get_dummies(df['Close Approach Date'])
encoded2=pd.get_dummies(df['Orbit Determination Date'])

col_names=['Earliest','Earlier','Later','Latest']
encoded2.columns=col_names

ndf=pd.concat([df, encoded1, encoded2], axis=1)

ndf['Hazardous'].replace(True, 1, inplace=True)
ndf['Hazardous'].replace(False, 1, inplace=True)

In [171]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

In [172]:
from sklearn.metrics import accuracy_score, confusion_matrix 
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

In [173]:
y=ndf['Hazardous']
X=ndf.drop(['Hazardous'], axis=1, inplace=False)

scaler=MinMaxScaler()
scaler.fit(X)
X_scaled=scaler.transform(X)

X_train, X_test, y_train, y_test=train_test_split(X_scaled, y, test_size=0.2, random_state=23)

In [174]:
def get_clf_eval(y_test,pred=None):
    
    accuracy = accuracy_score(y_test,pred)
    confusion = confusion_matrix(y_test,pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test,pred)
    f1=f1_score(y_test, pred)
    
    print('오차 행렬')
    print(confusion)
    print("정확도 : {0:.3f}, 정밀도 : {1:.3f}, 재현율 : {2:.3f}, f1:{3:.3f}".format(accuracy, precision, recall, f1))

In [175]:
rf_clf=RandomForestClassifier()

rf_clf.fit(X_train, y_train)
preds=rf_clf.predict(X_test)

get_clf_eval(y_test, preds)

오차 행렬
[[938]]
정확도 : 1.000, 정밀도 : 1.000, 재현율 : 1.000, f1:1.000
